In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 26 12:48:18 2025

@author: oujakusui
"""

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import CCA
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid', palette='muted')

# Step 1: Load RNA, CNA, and Clinical Data
# Update these paths based on your Google Drive location
rna_path = '/content/drive/MyDrive/brca_metabric/data_mrna_illumina_microarray.txt'
cna_path = '/content/drive/MyDrive/brca_metabric/data_cna.txt'
clinical_path = '/content/drive/MyDrive/brca_metabric_clinical_data.tsv'  # Use the correct file

# Load the data
rna = pd.read_csv(rna_path, sep='\t')
cna = pd.read_csv(cna_path, sep='\t')
clinical = pd.read_csv(clinical_path, sep='\t')

print("RNA Shape:", rna.shape)
print("CNA Shape:", cna.shape)
print("Clinical Shape:", clinical.shape)

# Step 2: Align Data by Sample IDs
# Check the column names in each dataset to confirm the ID column
print("\nRNA Columns (first 5):", rna.columns[:5])
print("CNA Columns (first 5):", cna.columns[:5])
print("Clinical Columns (first 5):", clinical.columns[:5])

# For RNA and CNA, the first few rows might be metadata; genes/features are columns
# Transpose to have samples as rows, genes as columns
# Drop 'Entrez_Gene_Id' if it exists and set 'Hugo_Symbol' as the column names
if 'Entrez_Gene_Id' in rna.columns:
    rna = rna.drop(columns=['Entrez_Gene_Id'])
rna = rna.set_index('Hugo_Symbol').T  # Transpose: samples as rows, genes as columns

if 'Entrez_Gene_Id' in cna.columns:
    cna = cna.drop(columns=['Entrez_Gene_Id'])
cna = cna.set_index('Hugo_Symbol').T  # Transpose: samples as rows, genes as columns

# Set index for clinical data using 'Sample ID' (correct for brca_metabric_clinical_data.tsv)
clinical = clinical.set_index('Sample ID')

# Align based on common sample IDs
common_ids = clinical.index.intersection(rna.index).intersection(cna.index)
rna = rna.loc[common_ids]
cna = cna.loc[common_ids]
clinical = clinical.loc[common_ids]

print("\nAfter Alignment:")
print("RNA Shape:", rna.shape)
print("CNA Shape:", rna.shape)
print("Clinical Shape:", clinical.shape)

# Step 3: Extract Labels for Evaluation
# Use 'Integrative Cluster' as the primary label
if 'Integrative Cluster' in clinical.columns:
    labels = clinical['Integrative Cluster'].dropna()
else:
    print("Integrative Cluster not found. Available columns:", clinical.columns)
    if 'Pam50 + Claudin-low subtype' in clinical.columns:
        labels = clinical['Pam50 + Claudin-low subtype'].dropna()  # Fallback label
    else:
        raise KeyError("Neither 'Integrative Cluster' nor 'Pam50 + Claudin-low subtype' found in clinical data.")

print("\nNumber of samples with labels:", len(labels))
print("Label Distribution:\n", labels.value_counts())

# Step 4: Handle Missing Values in RNA and CNA
# Drop columns (genes) with any missing values for simplicity
rna = rna.dropna(axis=1, how='any')
cna = cna.dropna(axis=1, how='any')

# Align again after dropping missing values
common_ids = clinical.index.intersection(rna.index).intersection(cna.index)
rna = rna.loc[common_ids]
cna = cna.loc[common_ids]
clinical = clinical.loc[common_ids]
labels = labels.loc[common_ids]

print("\nAfter Dropping Missing Values and Re-alignment:")
print("RNA Shape:", rna.shape)
print("CNA Shape:", cna.shape)
print("Clinical Shape:", clinical.shape)
print("Number of samples with labels after re-alignment:", len(labels))

# Step 5: Normalize Data
scaler = StandardScaler()
rna_scaled = scaler.fit_transform(rna)
cna_scaled = scaler.fit_transform(cna)

print("\nRNA scaled sample (first 5 rows, first 5 columns):")
print(rna_scaled[:5, :5])
print("CNA scaled sample (first 5 rows, first 5 columns):")
print(cna_scaled[:5, :5])


Mounted at /content/drive
RNA Shape: (20603, 1982)
CNA Shape: (22544, 2175)
Clinical Shape: (2509, 39)

RNA Columns (first 5): Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'MB-0362', 'MB-0346', 'MB-0386'], dtype='object')
CNA Columns (first 5): Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'MB-0000', 'MB-0039', 'MB-0045'], dtype='object')
Clinical Columns (first 5): Index(['Study ID', 'Patient ID', 'Sample ID', 'Age at Diagnosis',
       'Type of Breast Surgery'],
      dtype='object')

After Alignment:
RNA Shape: (1980, 20603)
CNA Shape: (1980, 20603)
Clinical Shape: (1980, 38)

Number of samples with labels: 1980
Label Distribution:
 Integrative Cluster
8       299
3       290
4ER+    260
10      226
5       190
7       190
9       146
1       139
6        85
4ER-     83
2        72
Name: count, dtype: int64

After Dropping Missing Values and Re-alignment:
RNA Shape: (1980, 20592)
CNA Shape: (1980, 22079)
Clinical Shape: (1980, 38)
Number of samples with labels after re-alignment: 1980

RNA sca

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Placeholder for loading METABRIC data (replace with your actual data loading)
# Assuming RNA (X), CNA (Y), and labels are provided as numpy arrays
# Example: rna_data, cna_data, labels = load_metabric_data()
# For demonstration, we'll simulate loading preprocessed data
rna_data = rna_scaled  # 1000 samples, 500 RNA features
cna_data = cna_scaled  # 1000 samples, 200 CNA features
labels = labels  # 10 cancer subtypes

# Preprocess data (assumed complete)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(rna_data)  # RNA data
Y_scaled = scaler.fit_transform(cna_data)  # CNA data

# Split into training and tuning sets (80-20 split)
X_train, X_tune, Y_train, Y_tune, labels_train, labels_tune = train_test_split(
    X_scaled, Y_scaled, labels, test_size=0.2, random_state=42
)

# Log data shapes after splitting
print(f"Training set shapes: X_train: {X_train.shape}, Y_train: {Y_train.shape}, labels_train: {labels_train.shape}")
print(f"Tuning set shapes: X_tune: {X_tune.shape}, Y_tune: {Y_tune.shape}, labels_tune: {labels_tune.shape}")

# SCCA Functions
def sweep(matrix, margin, stats, operation='/'):
    result_matrix = np.empty_like(matrix)
    if margin == 1:  # Column-wise
        for i in range(matrix.shape[1]):
            if operation == '/':
                result_matrix[:, i] = matrix[:, i] / stats[i]
            elif operation == '*':
                result_matrix[:, i] = matrix[:, i] * stats[i]
    elif margin == 0:  # Row-wise
        for i in range(matrix.shape[0]):
            if operation == '/':
                result_matrix[i, :] = matrix[i, :] / stats[i]
            elif operation == '*':
                result_matrix[i, :] = matrix[i, :] * stats[i]
    return result_matrix

def find_Omega(sigma_YX_hat, npairs, alpha=None, beta=None, y=None, x=None):
    n = y.shape[0]
    if npairs > 1:
        rho = alpha.T @ sigma_YX_hat @ beta
        omega = np.eye(n) - y @ alpha @ rho @ beta.T @ x.T / n
    else:
        omega = np.eye(n)
    return omega

def init0(sigma_YX_hat, sigma_X_hat, sigma_Y_hat, init_method, npairs, n):
    q, p = sigma_Y_hat.shape[1], sigma_X_hat.shape[1]
    if init_method == 'svd':
        u, _, v = np.linalg.svd(sigma_YX_hat, full_matrices=False)
        alpha_init = u[:, :npairs]
        beta_init = v.T[:, :npairs]
    elif init_method == 'random':
        alpha_init = np.random.normal(size=(q, npairs))
        beta_init = np.random.normal(size=(p, npairs))
    alpha_scale = np.diag(alpha_init.T @ sigma_Y_hat @ alpha_init)
    alpha_init = sweep(alpha_init, margin=1, stats=np.sqrt(alpha_scale), operation='/')
    beta_scale = np.diag(beta_init.T @ sigma_X_hat @ beta_init)
    beta_init = sweep(beta_init, margin=1, stats=np.sqrt(beta_scale), operation='/')
    return {'alpha_init': alpha_init, 'beta_init': beta_init}

def SCCA_solution(x, y, x_Omega, y_Omega, alpha0, beta0, standardize, lambda_alpha, lambda_beta, niter=100, eps=1e-4, column_index=0):
    n, q, p = x.shape[0], y.shape[1], x.shape[1]
    if alpha0.ndim > 1 and alpha0.shape[1] > 1:
        alpha0 = alpha0[:, column_index]
    if beta0.ndim > 1 and beta0.shape[1] > 1:
        beta0 = beta0[:, column_index]
    for i in range(niter):
        x0 = x_Omega @ beta0
        lasso_alpha = Lasso(alpha=lambda_alpha, fit_intercept=False)
        lasso_alpha.fit(y, x0)
        alpha1 = lasso_alpha.coef_
        if np.sum(np.abs(alpha1)) < eps:
            alpha0 = np.zeros(q)
            break
        alpha1_mask = np.abs(alpha1) > eps
        alpha1_scale = y[:, alpha1_mask] @ alpha1[alpha1_mask]
        alpha1 /= np.sqrt(alpha1_scale.T @ alpha1_scale / (n - 1))
        y0 = y_Omega @ alpha1
        lasso_beta = Lasso(alpha=lambda_beta, fit_intercept=False)
        lasso_beta.fit(x, y0)
        beta1 = lasso_beta.coef_
        if np.sum(np.abs(beta1)) < eps:
            beta0 = np.zeros(p)
            break
        beta1_scale = x[:, np.abs(beta1) > eps] @ beta1[np.abs(beta1) > eps]
        beta1 /= np.sqrt(beta1_scale.T @ beta1_scale / (n - 1))
        if np.sum(np.abs(alpha1 - alpha0)) < eps and np.sum(np.abs(beta1 - beta0)) < eps:
            break
        alpha0 = alpha1
        beta0 = beta1
    return {"alpha": alpha0, "beta": beta0, "niter": i+1}

def SCCA(x, y, lambda_alpha, lambda_beta, alpha_init=None, beta_init=None, niter=1000, npairs=1, init_method="svd", standardize=True, eps=1e-4):
    p, q, n = x.shape[1], y.shape[1], x.shape[0]
    x = StandardScaler().fit_transform(x) if standardize else x
    y = StandardScaler().fit_transform(y) if standardize else y
    sigma_YX_hat = np.cov(y.T, x.T)[:q, q:]
    sigma_X_hat = np.cov(x.T)
    sigma_Y_hat = np.cov(y.T)
    alpha = np.zeros((q, npairs))
    beta = np.zeros((p, npairs))
    if alpha_init is None:
        obj_init = init0(sigma_YX_hat, sigma_X_hat, sigma_Y_hat, init_method, npairs, n)
        alpha_init = obj_init['alpha_init']
        beta_init = obj_init['beta_init']
    for ipairs in range(npairs):
        omega = find_Omega(sigma_YX_hat, ipairs, alpha[:, :ipairs], beta[:, :ipairs], y, x)
        x_tmp = omega.dot(x)
        y_tmp = omega.T.dot(y)
        lambda_alpha0 = lambda_alpha if isinstance(lambda_alpha, float) else lambda_alpha[ipairs]
        lambda_beta0 = lambda_beta if isinstance(lambda_beta, float) else lambda_beta[ipairs]
        obj = SCCA_solution(x, y, x_tmp, y_tmp, alpha_init, beta_init, False, lambda_alpha0, lambda_beta0, niter, eps, ipairs)
        alpha[:, ipairs] = obj['alpha'].flatten()
        beta[:, ipairs] = obj['beta'].flatten()
    return {'alpha': alpha, 'beta': beta}

# Run SCCA with fixed lambda = 0.1
def run_scca_metabric(X_train, Y_train, X_tune, Y_tune, npairs=10):
    lambda_val = 0.1  # Fixed lambda value as requested
    print(f"Using fixed lambda: {lambda_val}\n")
    result = SCCA(X_train, Y_train, lambda_val, lambda_val, niter=100, npairs=npairs, init_method="svd")
    alpha_hat = result['alpha']
    beta_hat = result['beta']
    train_embedding = np.hstack((X_train @ beta_hat, Y_train @ alpha_hat))
    tune_embedding = np.hstack((X_tune @ beta_hat, Y_tune @ alpha_hat))
    print(f"Train embedding shape: {train_embedding.shape}")
    print(f"Tune embedding shape: {tune_embedding.shape}")
    return train_embedding, tune_embedding

# Generate Embeddings
train_embedding, tune_embedding = run_scca_metabric(X_train, Y_train, X_tune, Y_tune)

# Evaluation Functions
def compute_mi(embeddings, labels):
    print("Computing Mutual Information (MI)...")
    discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
    embeddings_discretized = discretizer.fit_transform(embeddings)
    mi_scores = [mutual_info_score(embeddings_discretized[:, i], labels) for i in range(embeddings_discretized.shape[1])]
    avg_mi = np.mean(mi_scores)
    print(f"Average MI: {avg_mi}")
    return avg_mi

# Compute Metrics
mi_scca = compute_mi(train_embedding, labels_train)
print("Performing KMeans clustering...")
kmeans_scca = KMeans(n_clusters=10, random_state=42)
scca_clusters = kmeans_scca.fit_predict(train_embedding)
scca_ari = adjusted_rand_score(labels_train, scca_clusters)
print(f"Adjusted Rand Index (ARI): {scca_ari}")
print("Performing SVM cross-validation...")
svm_scca = SVC(random_state=42)
scca_accuracy = cross_val_score(svm_scca, train_embedding, labels_train, cv=5).mean()
print(f"SVM Accuracy: {scca_accuracy}")

# Display Results
print("\nSCCA Evaluation Results:")
print(f"Mutual Information (MI): {mi_scca:.6f}")
print(f"Adjusted Rand Index (ARI, %): {scca_ari * 100:.2f}")
print(f"SVM Accuracy (%): {scca_accuracy * 100:.2f}")

Training set shapes: X_train: (1584, 20592), Y_train: (1584, 22079), labels_train: (1584,)
Tuning set shapes: X_tune: (396, 20592), Y_tune: (396, 22079), labels_tune: (396,)
Using fixed lambda: 0.1

